In [1]:
import pandas as pd
import numpy as np
import json
from math import log, floor
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./Data/preprocessed_data.csv')
with open('./Data/feature_config.json') as json_data:
  features = json.load(json_data)

features.keys()

len_columns = features['length']+ features['core_count']+ features['unique']+\
  features['blosum_sum']+ features['blosum']
len_fea = df[len_columns]
len_fea = len_fea.divide(len_fea.max())
df.drop(columns=len_columns, inplace = True)
df = df.join(len_fea)

origin_cols = df.columns
onehot_columns = features['hla']+features['core']+features['PFR']+features['cluster']
one_hot = pd.get_dummies(data=df, columns=onehot_columns)
one_hot.drop(columns = set(origin_cols)-set(onehot_columns), inplace = True)


In [3]:
def hla_preprocess(df):
  hla_encoder = LabelEncoder()
  #hla_encoder.fit(df['hla'])
  #np.save('Data/hla_encoder_classes.npy', hla_encoder.classes_)
  hla_encoder.classes_ = np.load('./Data/hla_encoder_classes.npy')
  df['hla'] = hla_encoder.transform(df['hla'])


  return df

v = pd.read_csv('./Data/vectors_data.csv')
v = hla_preprocess(v)
df = pd.merge(df, v, on = ['peptide', 'hla'], how = 'left')

columns = features['blosum']+features['unique']+features['core_tfidf']+[c for c in list(v.columns) if c not in ['peptide', 'hla']]+['aff']
cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

In [4]:
data = one_hot.join(df[columns]).drop_duplicates()
y = data['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)
data.drop(columns = 'aff', inplace = True)
data.fillna(0,inplace = True)
xtrain, xvalid, ytrain, yvalid = train_test_split(data, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)
result = {}
# Scale the data to (0,1) for Bayesyes
scl = MinMaxScaler()
scl.fit(xtrain)
xtrain_svd_scl = scl.transform(xtrain)
xvalid_svd_scl = scl.transform(xvalid)

In [5]:
pd.options.display.max_columns = 1200
data.head(2)

,hla_0,hla_2,hla_3,hla_5,hla_6,hla_7,hla_8,hla_9,hla_12,hla_13,hla_14,hla_17,hla_18,hla_19,hla_20,hla_21,hla_22,hla_23,hla_25,hla_26,hla_28,hla_29,hla_30,hla_31,hla_32,hla_33,hla_34,hla_36,hla_37,hla_38,hla_40,hla_41,hla_42,hla_43,hla_44,hla_45,hla_46,hla_47,hla_48,hla_49,hla_50,hla_51,hla_52,hla_53,hla_54,hla_55,hla_56,hla_59,hla_60,hla_62,hla_63,hla_64,hla_65,hla_67,hla_68,hla_71,hla_72,hla_73,hla_74,hla_75,hla_76,hla_77,hla_79,allele_type_0,allele_type_1,allele_type_2,allele_type_3,core_P1_1,core_P1_2,core_P1_3,core_P1_4,core_P1_5,core_P1_6,core_P1_7,core_P1_8,core_P1_9,core_P1_10,core_P1_11,core_P1_12,core_P1_13,core_P1_14,core_P1_15,core_P1_16,core_P1_17,core_P1_18,core_P1_19,core_P1_20,core_P2_1,core_P2_2,core_P2_3,core_P2_4,core_P2_5,core_P2_6,core_P2_7,core_P2_8,core_P2_9,core_P2_10,core_P2_11,core_P2_12,core_P2_13,core_P2_14,core_P2_15,core_P2_16,core_P2_17,core_P2_18,core_P2_19,core_P2_20,core_P3_1,core_P3_2,core_P3_3,core_P3_4,core_P3_5,core_P3_6,core_P3_7,core_P3_8,core_P3_9,core_P3_10,core_P3_11,core_P3_12,core_P3_13,core_P3_14,core_P3_15,core_P3_16,core_P3_17,core_P3_18,core_P3_19,core_P3_20,core_P4_1,core_P4_2,core_P4_3,core_P4_4,core_P4_5,core_P4_6,core_P4_7,core_P4_8,core_P4_9,core_P4_10,core_P4_11,core_P4_12,core_P4_13,core_P4_14,core_P4_15,core_P4_16,core_P4_17,core_P4_18,core_P4_19,core_P4_20,core_P5_1,core_P5_2,core_P5_3,core_P5_4,core_P5_5,core_P5_6,core_P5_7,core_P5_8,core_P5_9,core_P5_10,core_P5_11,core_P5_12,core_P5_13,core_P5_14,core_P5_15,core_P5_16,core_P5_17,core_P5_18,core_P5_19,core_P5_20,core_P6_1,core_P6_2,core_P6_3,core_P6_4,core_P6_5,core_P6_6,core_P6_7,core_P6_8,core_P6_9,core_P6_10,core_P6_11,core_P6_12,core_P6_13,core_P6_14,core_P6_15,core_P6_16,core_P6_17,core_P6_18,core_P6_19,core_P6_20,core_P7_1,core_P7_2,core_P7_3,core_P7_4,core_P7_5,core_P7_6,core_P7_7,core_P7_8,core_P7_9,core_P7_10,core_P7_11,core_P7_12,core_P7_13,core_P7_14,core_P7_15,core_P7_16,core_P7_17,core_P7_18,core_P7_19,core_P7_20,core_P8_1,core_P8_2,core_P8_3,core_P8_4,core_P8_5,core_P8_6,core_P8_7,core_P8_8,core_P8_9,core_P8_10,core_P8_11,core_P8_12,core_P8_13,core_P8_14,core_P8_15,core_P8_16,core_P8_17,core_P8_18,core_P8_19,core_P8_20,core_P9_1,core_P9_2,core_P9_3,core_P9_4,core_P9_5,core_P9_6,core_P9_7,core_P9_8,core_P9_9,core_P9_10,core_P9_11,core_P9_12,core_P9_13,core_P9_14,core_P9_15,core_P9_16,core_P9_17,core_P9_18,core_P9_19,core_P9_20,LPFR_P1_0,LPFR_P1_1,LPFR_P1_2,LPFR_P1_3,LPFR_P1_4,LPFR_P1_5,LPFR_P1_6,LPFR_P1_7,LPFR_P1_8,LPFR_P1_9,LPFR_P1_10,LPFR_P1_11,LPFR_P1_12,LPFR_P1_13,LPFR_P1_14,LPFR_P1_15,LPFR_P1_16,LPFR_P1_17,LPFR_P1_18,LPFR_P1_19,LPFR_P1_20,LPFR_P2_0,LPFR_P2_1,LPFR_P2_2,LPFR_P2_3,LPFR_P2_4,LPFR_P2_5,LPFR_P2_6,LPFR_P2_7,LPFR_P2_8,LPFR_P2_9,LPFR_P2_10,LPFR_P2_11,LPFR_P2_12,LPFR_P2_13,LPFR_P2_14,LPFR_P2_15,LPFR_P2_16,LPFR_P2_17,LPFR_P2_18,LPFR_P2_19,LPFR_P2_20,LPFR_P3_0,LPFR_P3_1,LPFR_P3_2,LPFR_P3_3,LPFR_P3_4,LPFR_P3_5,LPFR_P3_6,LPFR_P3_7,LPFR_P3_8,LPFR_P3_9,LPFR_P3_10,LPFR_P3_11,LPFR_P3_12,LPFR_P3_13,LPFR_P3_14,LPFR_P3_15,LPFR_P3_16,LPFR_P3_17,LPFR_P3_18,LPFR_P3_19,LPFR_P3_20,RPFR_P1_0,RPFR_P1_1,RPFR_P1_2,RPFR_P1_3,RPFR_P1_4,RPFR_P1_5,RPFR_P1_6,RPFR_P1_7,RPFR_P1_8,RPFR_P1_9,RPFR_P1_10,RPFR_P1_11,RPFR_P1_12,RPFR_P1_13,RPFR_P1_14,RPFR_P1_15,RPFR_P1_16,RPFR_P1_17,RPFR_P1_18,RPFR_P1_19,RPFR_P1_20,RPFR_P2_0,RPFR_P2_1,RPFR_P2_2,RPFR_P2_3,RPFR_P2_4,RPFR_P2_5,RPFR_P2_6,RPFR_P2_7,RPFR_P2_8,RPFR_P2_9,RPFR_P2_10,RPFR_P2_11,RPFR_P2_12,RPFR_P2_13,RPFR_P2_14,RPFR_P2_15,RPFR_P2_16,RPFR_P2_17,RPFR_P2_18,RPFR_P2_19,RPFR_P2_20,RPFR_P3_0,RPFR_P3_1,RPFR_P3_2,RPFR_P3_3,RPFR_P3_4,RPFR_P3_5,RPFR_P3_6,RPFR_P3_7,RPFR_P3_8,RPFR_P3_9,RPFR_P3_10,RPFR_P3_11,RPFR_P3_12,RPFR_P3_13,RPFR_P3_14,RPFR_P3_15,RPFR_P3_16,RPFR_P3_17,RPFR_P3_18,RPFR_P3_19,RPFR_P3_20,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cluster_18,cluster_19,cluster_20,cluster_21,cluster_22,cluster_23,cluster_24,cluster_25,cluster_2

In [6]:
data.shape

(974576, 973)

In [7]:
zxc = pd.read_csv('./feature_selected.csv')

In [8]:
zxc.head()

,hla_0,hla_2,hla_3,hla_4,hla_5,hla_6,hla_7,hla_8,hla_9,hla_12,hla_13,hla_14,hla_17,hla_18,hla_19,hla_21,hla_22,hla_23,hla_25,hla_28,hla_29,hla_30,hla_31,hla_32,hla_33,hla_34,hla_36,hla_37,hla_38,hla_39,hla_40,hla_41,hla_42,hla_43,hla_44,hla_45,hla_46,hla_47,hla_48,hla_49,hla_50,hla_51,hla_52,hla_53,hla_54,hla_55,hla_56,hla_57,hla_59,hla_60,hla_62,hla_63,hla_64,hla_65,hla_67,hla_68,hla_71,hla_72,hla_73,hla_74,hla_75,hla_76,hla_77,hla_79,allele_type_0,allele_type_1,allele_type_2,allele_type_3,core_P1_1,core_P1_2,core_P1_3,core_P1_4,core_P1_5,core_P1_6,core_P1_7,core_P1_8,core_P1_9,core_P1_10,core_P1_11,core_P1_12,core_P1_13,core_P1_14,core_P1_15,core_P1_16,core_P1_17,core_P1_18,core_P1_19,core_P1_20,core_P2_1,core_P2_2,core_P2_3,core_P2_4,core_P2_5,core_P2_6,core_P2_7,core_P2_8,core_P2_9,core_P2_10,core_P2_11,core_P2_12,core_P2_13,core_P2_14,core_P2_15,core_P2_16,core_P2_17,core_P2_18,core_P2_19,core_P2_20,core_P3_1,core_P3_2,core_P3_3,core_P3_4,core_P3_5,core_P3_6,core_P3_7,core_P3_8,core_P3_9,core_P3_10,core_P3_11,core_P3_12,core_P3_13,core_P3_14,core_P3_15,core_P3_16,core_P3_17,core_P3_18,core_P3_19,core_P3_20,core_P4_1,core_P4_2,core_P4_3,core_P4_4,core_P4_5,core_P4_6,core_P4_7,core_P4_8,core_P4_9,core_P4_10,core_P4_11,core_P4_12,core_P4_13,core_P4_14,core_P4_15,core_P4_16,core_P4_17,core_P4_18,core_P4_19,core_P4_20,core_P5_1,core_P5_2,core_P5_3,core_P5_4,core_P5_5,core_P5_6,core_P5_7,core_P5_8,core_P5_9,core_P5_10,core_P5_11,core_P5_12,core_P5_13,core_P5_14,core_P5_15,core_P5_16,core_P5_17,core_P5_18,core_P5_19,core_P5_20,core_P6_1,core_P6_2,core_P6_3,core_P6_4,core_P6_5,core_P6_6,core_P6_7,core_P6_8,core_P6_9,core_P6_10,core_P6_11,core_P6_12,core_P6_13,core_P6_14,core_P6_15,core_P6_16,core_P6_17,core_P6_18,core_P6_19,core_P6_20,core_P7_1,core_P7_2,core_P7_3,core_P7_4,core_P7_5,core_P7_6,core_P7_7,core_P7_8,core_P7_9,core_P7_10,core_P7_11,core_P7_12,core_P7_13,core_P7_14,core_P7_15,core_P7_16,core_P7_17,core_P7_18,core_P7_19,core_P7_20,core_P8_1,core_P8_2,core_P8_3,core_P8_4,core_P8_5,core_P8_6,core_P8_7,core_P8_8,core_P8_9,core_P8_10,core_P8_11,core_P8_12,core_P8_13,core_P8_14,core_P8_15,core_P8_16,core_P8_17,core_P8_18,core_P8_19,core_P8_20,core_P9_1,core_P9_2,core_P9_3,core_P9_4,core_P9_5,core_P9_6,core_P9_7,core_P9_8,core_P9_9,core_P9_10,core_P9_11,core_P9_12,core_P9_13,core_P9_14,core_P9_15,core_P9_16,core_P9_17,core_P9_18,core_P9_19,core_P9_20,LPFR_P1_0,LPFR_P1_1,LPFR_P1_2,LPFR_P1_3,LPFR_P1_4,LPFR_P1_5,LPFR_P1_6,LPFR_P1_7,LPFR_P1_8,LPFR_P1_9,LPFR_P1_10,LPFR_P1_11,LPFR_P1_12,LPFR_P1_13,LPFR_P1_14,LPFR_P1_15,LPFR_P1_16,LPFR_P1_17,LPFR_P1_18,LPFR_P1_19,LPFR_P1_20,LPFR_P2_0,LPFR_P2_1,LPFR_P2_2,LPFR_P2_3,LPFR_P2_4,LPFR_P2_5,LPFR_P2_6,LPFR_P2_7,LPFR_P2_8,LPFR_P2_9,LPFR_P2_10,LPFR_P2_11,LPFR_P2_12,LPFR_P2_13,LPFR_P2_14,LPFR_P2_15,LPFR_P2_16,LPFR_P2_17,LPFR_P2_18,LPFR_P2_19,LPFR_P2_20,LPFR_P3_0,LPFR_P3_1,LPFR_P3_2,LPFR_P3_3,LPFR_P3_4,LPFR_P3_5,LPFR_P3_6,LPFR_P3_7,LPFR_P3_8,LPFR_P3_9,LPFR_P3_10,LPFR_P3_11,LPFR_P3_12,LPFR_P3_13,LPFR_P3_14,LPFR_P3_15,LPFR_P3_16,LPFR_P3_17,LPFR_P3_18,LPFR_P3_19,LPFR_P3_20,RPFR_P1_0,RPFR_P1_1,RPFR_P1_2,RPFR_P1_3,RPFR_P1_4,RPFR_P1_5,RPFR_P1_6,RPFR_P1_7,RPFR_P1_8,RPFR_P1_9,RPFR_P1_10,RPFR_P1_11,RPFR_P1_12,RPFR_P1_13,RPFR_P1_14,RPFR_P1_15,RPFR_P1_16,RPFR_P1_17,RPFR_P1_18,RPFR_P1_19,RPFR_P1_20,RPFR_P2_0,RPFR_P2_1,RPFR_P2_2,RPFR_P2_3,RPFR_P2_4,RPFR_P2_5,RPFR_P2_6,RPFR_P2_7,RPFR_P2_8,RPFR_P2_9,RPFR_P2_10,RPFR_P2_11,RPFR_P2_12,RPFR_P2_13,RPFR_P2_14,RPFR_P2_15,RPFR_P2_16,RPFR_P2_17,RPFR_P2_18,RPFR_P2_19,RPFR_P2_20,RPFR_P3_0,RPFR_P3_1,RPFR_P3_2,RPFR_P3_3,RPFR_P3_4,RPFR_P3_5,RPFR_P3_6,RPFR_P3_7,RPFR_P3_8,RPFR_P3_9,RPFR_P3_10,RPFR_P3_11,RPFR_P3_12,RPFR_P3_13,RPFR_P3_14,RPFR_P3_15,RPFR_P3_16,RPFR_P3_17,RPFR_P3_18,RPFR_P3_19,RPFR_P3_20,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cluster_18,cluster_19,cluster_20,cluster_21,cluster_22,cluster_23,cluster_24,cluster_25,clu